For faster development speed, type `Ctrl + M, H` to view Google Colab keyboard shortcuts.

### Getting Started


#### Mounting your Google Drive

In [18]:
from google.colab import drive

"""Running this cell will generate a message asking you 
to click on a link where you'll obtain an authorization code.

Paste that authorization code into the text box that appears below.
"""
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [29]:
!cd "/content/gdrive/MyDrive/" && ls # Displays directories in MyDrive/

 ApTSi		    Education	'School-Related Planning'   地位を申し込む
'Colab Notebooks'   Other	 SCIP
 Data		    Recordings	 temp


In [3]:
import torch

In [9]:
print(torch.cuda.device_count()) # number of GPU devices available
print(torch.cuda.get_device_name(0)) # name of device
device = torch.device('cuda:0') # sets the device to the cuda at index 0
device

1
Tesla K80


device(type='cuda', index=0)

In [7]:
torch.device('cuda:0')

device(type='cuda', index=0)